### CH06 Getting More out of our Objects With Descriptors

:파이썬의 고급기능 디스크립터 다른언어에는 생소한 개념

#### 디스크립터 개요

디스크립터 메커니즘(descriptor mechanism)
- 최소 두 개의 클래스가 필요
- 디스크립터 프로토콜 중 최소 한개 이상 클래스 속성으로 포함해야 함

(python 3.6기준 디스크립터 프로토콜)

- __get__
- __set__
- __delete__
- __set_name__

Tip : 디스크립터 객체는 항상 클래스 속성으로 선언해야 한다!

In [1]:
""" 디스크립터는 __get__ 매직 메서드의 결과를 반환함 """

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner): # owner의 인스턴스와 클래스를 받음
        if instance is None:
            return self
        logger.info(
            "Call: %s.__get__(%r, %r)", 
            self.__class__.__name__,
            instance,
            owner,
        )
        return instance


class ClientClass:
    descriptor = DescriptorClass()

In [2]:
client = ClientClass()
client.descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103448dc0>, <class '__main__.ClientClass'>)


In [3]:
client.descriptor is client

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103448dc0>, <class '__main__.ClientClass'>)


True

#### 디스크립터 프로토콜의 메서드 검색

 _ _ get _ _(self, isinstance, owner)

- owner가 있는 경우는 클래스에서 직접 호출하는 경우 isinstance가 None이기 떄문

In [4]:
# descripters_methods_1.py

""" 디스크립터를 인스턴스로 호출할 때와 클래스 속성으로 호출할 때의 차이점 """

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self, owner
        logger.info("Call: %s.__get__(%r, %r)",
                    self.__class__.__name__, instance, owner)
        return instance
    
class ClientClass:
    descriptor = DescriptorClass()

In [7]:
ClientClass.descriptor

(<__main__.DescriptorClass at 0x103463e80>, __main__.ClientClass)

In [8]:
ClientClass().descriptor # instance가 None이므로 self를 반환

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103463850>, <class '__main__.ClientClass'>)


##### _ _ set _ _(self, instance, value)

- 디스크립터에 값을 할당할 때 호출되며, _ _set_ _( ) 메서드를 구현한 디스크립터에 대해서만 활성화
-  _ _set_ _ 메서드를구현했는지 반드시 확인하여 부작용이생기지않도록 주의해야한다.

In [9]:
from typing import Callable, Any # 형 힌트 지원
                                 # Callable[[Arg1Type, Arg2Type], ReturnType] 

class Validation: # 검증함수와 에러 메세지만을 가지고 있고 value는 __set__ 호출 시 전달받음
    """A configurable validation callable."""

    def __init__(
        self, validation_function: Callable[[Any], bool], error_msg: str 
    ) -> None:
        self.validation_function = validation_function
        self.error_msg = error_msg

    def __call__(self, value): # client.descriptor = 42 에서 42
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")


class Field:
    """A class attribute with validation functions configured over it."""
    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value): # client.descriptor = 42 처럼 대입 시 실행됨
        self.validate(value) # 여기서 raise ValueError 안된다면 다음 줄에서 대입됨
        instance.__dict__[self._name] = value


class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "is not a number"),
        Validation(lambda x: x >= 0, "is not >= 0"),
    )

In [10]:
client = ClientClass()
client.descriptor = 42
print(client.descriptor)

client.descriptor = -42

42


ValueError: -42 is not >= 0

In [11]:
client.descriptor = "string"

ValueError: 'string' is not a number

._s..et_0메서드가@property,setter가하던 일을대신

##### _ _ delete _ _(self, instance)

self 는 descriptor 속성을 나타내며, instance 는 client 를 나타낸다.

In [2]:
# descripters_methods_3.py

class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name} can't be set to None")
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {user!s} doesn't have {self.permission_required} "
                "permission"
            )


class User:
    """admin 권한이 있는 유저만 이메일 주소를 지울 수 있음"""

    email = ProtectedAttribute(requires_role="admin")

    def __init__(
        self, username: str, email: str, permission_list: list = None
    ) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username

In [3]:
admin = User("root", "root@d.com", ["admin"])
user = User("user", "user@d.com", ["email", "helpdesk"])

In [4]:
del admin.email
admin.email is None

True

In [5]:
del user.email

ValueError: User user doesn't have admin permission

_ _ set_name_ _ (self, owner, name)

- 디스크립터가 처리하려는 속성의 이름을 알려줌
- 속성의 이름은 _ _dict_ _ 에서 _ _get_ _ 과 _ _set_ _ 메서드로 읽고 쓸 때 사용

In [6]:
class DescriptorWithName:
    def __init__(self, name: str = None)-> None:
        self.name = name
    
    def __set_name__(self, owner, name):
        self.name = self.name or name
        
class ClientClass:
    descripter = DescriptorWithName()

In [7]:
client = ClientClass()
client.descripter.name

'descripter'

#### 디스크립터 유형

- _ _set_ _, _ _delete_ _메서드를 구현했다면 data desriptor 구분됨

- _ _get_ _만 구현했다면 non-data descriptor

##### non-data descripter


In [8]:
class NonDataDescriptor:
    def __get__(self, isinstance, owner):
        if isinstance is None:
            return self
        return 42

class ClientClass:
    descriptor = NonDataDescriptor()

In [9]:
client = ClientClass()
client.descriptor

42

In [10]:
client.descriptor = 43
client.descriptor

43

In [12]:
vars(client)

# 역주 : vars는 파이선의 내장함수로 클래스의 속성을 저장한느 사전인 __dict__를 반환

{'descriptor': 43}

In [13]:
del client.descriptor
client.descriptor  
#  non-data descriptor의 경우 내부 딕셔너리보다 후순위로 검색

42

##### Data Descriptor

- _ _set_ _() object._ _dict_ _보다 높은 우선순위

- 디스크립터의 set에서 setattr()이나 할당 표현식을 직접 사용하면  
 무한루프가 발생하므로 주의가 필요함

In [15]:
class DataDescriptor:
    
    def __init__(self):
        self._name = None
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42
    
    def __set__(self, instance, value):
        instance.__dict__[self._name] = value
        
    def __set_name__(self, owner, name):
        self._name = name
        

class ClientClass:
    descriptor = DataDescriptor()
    
    

In [16]:
client = ClientClass()
client.descriptor

42

In [17]:
client.descriptor = 99
client.descriptor

42

In [18]:
vars(client)

{'descriptor': 99}

In [19]:
client.__dict__['descriptor']

99

이렇게 되는 이유 _ _set_ _() 메서드가 호출되면 사전에 값을 설정  
data descriptor 속성 조회시 _ _dict_ _ 대신 클래스의 descriptor 먼저 조회

Tip: descriptor의 _ _set_ _ 메서드에서 setattr()이나 할당 표현식을 직접 사용하면 안된다.  
무한루프가 발생


역주: instance.descriptor = value와 같은 할당 표현식도 같은 이유로 무한루프 유발

In [ ]:
#### Descriptor 실전


##### 디스크립터를 사용하지 않은 예

In [20]:
"""Clean Code in Python - Chapter 6: Descriptors
> A Pythonic Implementation
"""
import time

class Traveller:
    def __init__(self, name, current_city):
        self.name = name
        self._current_city = current_city
        self._cities_visited = [current_city]

    @property
    def current_city(self):
        return self._current_city

    @current_city.setter
    def current_city(self, new_city):
        if new_city != self._current_city:
            self._cities_visited.append(new_city)
        self._current_city = new_city

    @property
    def cities_visited(self):
        return self._cities_visited


In [21]:
alice = Traveller("Alice", "Barcelona")
alice.current_city = "Paris"
alice.current_city = "Brussels"
alice.current_city = "Amsterdam"
alice.cities_visited


['Barcelona', 'Paris', 'Brussels', 'Amsterdam']

#### 이상적인 구현 방법

In [22]:
# descriptors_pythonic_2.py

class HistoryTracedAttribute:
    """trace_attribute_name 값에 따라서 이를 추적하는 클래스"""
    def __init__(self, trace_attribute_name: str) -> None:
        self.trace_attribute_name = trace_attribute_name  # [1]
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value

    def _track_change_in_value_for_instance(self, instance, value):
        self._set_default(instance)  # [2]
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)

    def _needs_to_track_change(self, instance, value) -> bool:
        """값의 변화를 추적할지 여부를 결정하는 메서드

        추적 조건:
            * 첫 번째 데이터
            * current_value 와 같지 않은 경우
        """
        try:
            current_value = instance.__dict__[self._name]
        except KeyError:  # [3]
            return True
        return value != current_value  # [4]

    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, [])  # [6]


class Traveller:
    current_city = HistoryTracedAttribute("cities_visited")  # [1]

    def __init__(self, name, current_city):
        self.name = name
        self.current_city = current_city  # [5]

In [23]:
alice = Traveller("Alice", "Barcelona")
alice.current_city = "Paris"
alice.current_city = "Brussels"
alice.current_city = "Brussels"
alice.current_city = "Amsterdam"
alice.cities_visited

['Barcelona', 'Paris', 'Brussels', 'Amsterdam']

In [24]:
alice.current_city

'Amsterdam'

#### 다른 형태의 디스크립터

- 전역 상태 공유 이슈

In [25]:
class SharedDataDescriptor:
    def __init__(self, initial_value):
        self.value = initial_value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.value

    def __set__(self, instance, value):
        self.value = value


class ClientClass:
    descriptor = SharedDataDescriptor("first value")

In [27]:
client1 = ClientClass()
client1.descriptor

'first value'

In [28]:
client2 = ClientClass()
client2.descriptor

'first value'

In [29]:
client2.descriptor = "value for client 2"
client2.descriptor

'value for client 2'

In [30]:
client1.descriptor

'value for client 2'

##### 객체의 사전에 접근하기

디스크립터 객체의 사전 __dict__에 값을 저장하고 조회한다.  
 | 항상 인스턴스의 __dict__속성에서 데이터를 저장하고 조회한다.

- 약한 참조 사용

In [31]:
from weakref import WeakKeyDictionary


class DescriptorClass:
    def __init__(self, initial_value):
        self.value = initial_value
        self.mapping = WeakKeyDictionary()

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.mapping.get(instance, self.value)

    def __set__(self, instance, value):
        self.mapping[instance] = value


class ClientClass:
    descriptor = DescriptorClass("default value")

#### 디스크립터에 대한 고려사항

클래스 데코레이터 피하기

In [32]:

from datetime import datetime
from functools import partial
from typing import Any, Callable


class BaseFieldTransformation:
    """Base class to define descriptors that convert values."""

    def __init__(self, transformation: Callable[[Any, str], str]) -> None:
        self._name = None
        self.transformation = transformation

    def __get__(self, instance, owner):
        if instance is None:
            return self
        raw_value = instance.__dict__[self._name]
        return self.transformation(raw_value)

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, instance, value):
        instance.__dict__[self._name] = value


ShowOriginal = partial(BaseFieldTransformation, transformation=lambda x: x) # transformation이 자동 설정됨
HideField = partial(BaseFieldTransformation, transformation=lambda x: "**redacted**")
FormatTime = partial(
    BaseFieldTransformation,
    transformation=lambda ft: ft.strftime("%Y-%m-%d %H:%M"),
)


class LoginEvent:
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()

    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp

    def serialize(self):
        return {
            "username": self.username,
            "password": self.password,
            "ip": self.ip,
            "timestamp": self.timestamp,
        }


class BaseEvent:
    """Abstract the serialization and the __init__"""

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def serialize(self):
        return {
            attr: getattr(self, attr) for attr in self._fields_to_serialize()
        }

    def _fields_to_serialize(self):
        for attr_name, value in vars(self.__class__).items():
            if isinstance(value, BaseFieldTransformation): # '__module__, __doc__ 등은 제외하고 변환함
                yield attr_name


class NewLoginEvent(BaseEvent):
    """A class that takes advantage of the base to only define the fields."""

    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()

In [33]:
le = NewLoginEvent(username = "usr",
                   password = "secret password",
                   ip = "127.0.0.1",
                   timestamp = datetime(2016, 7, 20, 15, 45))

In [34]:
le.serialize()

{'username': 'usr',
 'password': '**redacted**',
 'ip': '127.0.0.1',
 'timestamp': '2016-07-20 15:45'}

##### 파이썬 내부에서의 디스크립터 활용

In [35]:
""" 함수와 메서드 """

class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출, 인자는 {arg1}과 {arg2}")
        
class MyClass:
    method = Method("Internal call")

In [36]:
instance = MyClass()
Method("External call")(instance, "1st", "2nd")

External call: <__main__.MyClass object at 0x1036a87f0> 호출, 인자는 1st과 2nd


In [37]:

instance.method("1st", "2nd")

TypeError: __call__() missing 1 required positional argument: 'arg2'

In [38]:

""" 함수와 메서드, get을 구현하여 클래스 메소드로도 사용할 수 있도록 개선 """
from types import MethodType

class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출, 인자는 {arg1}과 {arg2}")
        
    def __get__(self, myclass_inst, myclass_cls):
        if myclass_inst == None:
            return self
        else:
            return MethodType(self, myclass_inst)
        
class MyClass:
    method = Method("Internal call")

In [39]:
instance = MyClass()
Method("External call")(instance, "1st", "2nd")
instance.method("1st", "2nd")

External call: <__main__.MyClass object at 0x1036cbb50> 호출, 인자는 1st과 2nd
Internal call: <__main__.MyClass object at 0x1036cbb50> 호출, 인자는 1st과 2nd
